<div style="font-size:13px">

### 🇰🇷 [KR]

---

#### 📌 개요

각 경기(`matchid`)에 **Pick된 챔피언 목록을 저장**하는 작업입니다.

- `.rofl` 리플레이 파일이 아닌  
- **CSV 파일**에 있는 `matchid` 목록을 읽어  
- 각 경기에 **참여한 챔피언 이름들**을 추출하여 저장합니다.

</div>



In [1]:
import os, json, csv
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

from autoLeague.replays.scraper import ReplayScraper
from autoLeague.dataset.riotapi import RiotAPI

load_dotenv()  
API_KEY = os.environ["API_KEY"]

# ── 공통 경로 설정(Set common base path) ─────────────────────────────────────
USER_HOME = Path(os.environ["USERPROFILE"])    # C:\Users\<계정명>

GAME_DIR        = Path(r"C:\Riot Games\League of Legends\Game")
REPLAY_DIR      = str(USER_HOME / "Documents" / "League of Legends" / "Replays")
SAVE_DIR        = str(USER_HOME / "Desktop" / "storage")
SCRAPER_DIR     = str(USER_HOME / "Desktop" / "pyLoL" / "pyLoL" / "autoLeague" / "replays")
MATCH_INFO_DIR  = str(USER_HOME / "Desktop" / "pyLoL" / "data" / "match_info")

ra = RiotAPI(api_key=API_KEY)
rs = ReplayScraper(game_dir=GAME_DIR, replay_dir=REPLAY_DIR, save_dir=SAVE_DIR, scraper_dir=SCRAPER_DIR, replay_speed=27, region="KR")

현재 있는 리플레이 갯수 :  6893


<div style="font-size:13px">

### 🇰🇷 [KR]

---

#### 📌 개요

이전 단계에서 저장한 **여러 리그의 `matchid` 목록들을 하나의 CSV 파일로 병합**하는 작업입니다.

- 다양한 리그에서 수집한 `matchid` CSV들을  
- **하나의 통합 CSV 파일로 정리**합니다.

---

### 🇺🇸 [EN]

---

#### 📌 Overview

This step **merges the `matchid` lists collected from multiple leagues into a single CSV file**.

- Combines `matchid` CSV files from various leagues  
- into **one consolidated CSV file**.

</div>


In [ ]:
# 파일 경로 리스트(List of file paths)
file_paths = [
    "matchids/matchids_CHALLENGER_I_patch25_13.csv",
    "matchids/matchids_GRANDMASTER_I_patch25_13.csv"
]

# 모든 파일 읽어서 하나로 합치기(Read all files and merge into one DataFrame)
df_list = [pd.read_csv(path) for path in file_paths]
merged_df = pd.concat(df_list, ignore_index=True)

# 중복 제거 : match_id 컬럼 기준 (Remove duplicates (based on the 'match_id' column))
if 'match_id' in merged_df.columns:
    merged_df = merged_df.drop_duplicates(subset='match_id')
else:
    merged_df = merged_df.drop_duplicates()

# 결과 저장(Save the result)
merged_df.to_csv("data/match_champions(25.13).csv", index=False)

<div style="font-size:13px">

### 🇰🇷 [KR]

---

#### 🔀 OPTION 1: `[2] download_replays.ipynb` 단계를 건너뛴 경우

- `.rofl` 리플레이 파일이 아닌  
- **CSV 파일에 있는 `matchid` 목록을 읽어**,  
- 각 경기에 **참여한 챔피언 이름들을 저장**합니다.

---

### 🇺🇸 [EN]

---

#### 🔀 OPTION 1: If you skipped step `[2] download_replays.ipynb`

- Instead of using `.rofl` replay files,  
- this step **reads `matchid` values from a CSV file** and  
- **stores the names of champions who participated** in each match.

</div>


In [ ]:
ra.save_champnames_from_matches_without_rofl(input_csv_path='data/match_champions(25.13).csv',
                                             output_csv_path='data/match_champions_dict(25.13).csv') 

<div style="font-size:13px">

### 🇰🇷 [KR]

---

#### 🔀 OPTION 2: `[2] download_replays.ipynb` 단계를 완료한 경우

- `.rofl` 리플레이 파일들을 **직접 읽어서**,  
- 각 경기에 **참여한 챔피언 이름들을 저장**합니다.

---

### 🇺🇸 [EN]

---

#### 🔀 OPTION 2: If you completed step `[2] download_replays.ipynb`

- Reads the `.rofl` replay files directly  
- and **records the champion names** for each match.

</div>


In [ ]:
ra.save_champnames_from_matches_with_rofl('data', 'match_champions_dict(25.13).csv', REPLAY_DIR) 
# 리플레이 파일이 있는 디렉토리 (예시 경로) : replay_dir = r'C:\Users\username\Documents\League of Legends\Replays'
# CSV를 저장할 디렉토리 : csv_save_folder_dir = 'data'

<div style="font-size:13px">

### 🇰🇷 [KR]

---

#### 🧹 STEP 2: 탐지 불가 또는 인게임 초상화가 잦은 챔피언 제외

- YOLO-11 기반 **탐지 모델이 제대로 인식하지 못하는 챔피언들**  
  (→ 학습 데이터 문제)  
- 또는 **패시브 / 스킬로 초상화가 자주 바뀌는 챔피언들**  
  (→ 인게임 구조 문제)  
을 **데이터셋에서 제거하는 단계**입니다.

---

### 🇺🇸 [EN]

---

#### 🧹 STEP 2: Filtering out problematic champion classes

- Excludes champions that the **YOLO-11–based model cannot reliably detect**  
  (due to training data limitations),  
- and champions whose **in-game portraits frequently change**  
  (due to passives or skill transformations).  
This step filters them **out of the dataset**.

</div>


In [ ]:
# ── 설정값 ─────────────────────────────────────────────
CSV_IN  = "data/match_champions_dict(25.13).csv"   # 원본 CSV 
CSV_OUT = "data/filtered_matchids(25.13).csv"              # 결과 저장 경로
EXCLUDED_CHAMPIONS = {"Neeko", "Viego", "Yuumi", "Amumu",
                      "Kayle", "Nilah", "Shyvana"}  # 제외 대상 챔피언
# ────────────── ───────────────────────────────────────

ra.filter_matches_by_excluded_champions(CSV_IN, CSV_OUT, EXCLUDED_CHAMPIONS)

<div style="font-size:13px">

### 🇰🇷 [KR]

---

#### 🎯 STEP 3 (선택): 10명 평균 점수 기반 상위 경기만 추출

- **유저 10명의 평균 점수 기준**으로  
- **상위 N개 경기만 필터링**하고 싶다면  
- 이 단계를 적용하면 됩니다.

---

### 🇺🇸 [EN]

---

#### 🎯 STEP 3 (Optional): Filter top N matches by average player score

- If you want to **select only the top N matches**  
- based on the **average score of all 10 players**,  
- you can **apply this optional filtering step**.

</div>


In [ ]:
MATCH_AVGSCR_CSV_PATH = "data/match_avg_score(25.13).csv"

### [1] match_champ_dict CSV 파일이 있는 경우 사용 > match_info 폴더에 개별적으로 저장.
ra.save_summoner_leagueinfo_from_csv(csv_in=CSV_IN, save_folder=MATCH_INFO_DIR, region="KR", queue_type="RANKED_SOLO_5x5")# 기본값queue_type="RANKED_SOLO_5x5"

### match_champ_dict CSV 파일이 없는 경우 사용
# ra.save_summoner_leagueinfo_of_replays(replay_dir = REPLAY_DIR, save_folder = MATCH_INFO_DIR,region="KR")# 기본값queue_type="RANKED_SOLO_5x5"

### [2] 각 경기 유저 10명의 평균 점수 계산하여 csv 에 저장
### Calculate the average score of the 10 players in each match and save the results to a CSV file.

ra.save_match_avg_score_from_json(match_info_dir=MATCH_INFO_DIR, output_csv_path=MATCH_AVGSCR_CSV_PATH )
matchid_list = pd.read_csv(MATCH_AVGSCR_CSV_PATH, usecols=["match_id"]).values.flatten().tolist()

### [3] 각 경기에서 각 시간대(분 단위)별로 킬 이벤트가 threshold 이상인 시간대만만 추출하여 json 파일로 저장
ra.save_kill_events_from_matchlist(matchids=matchid_list, output_json_path="data/kill_events_timeline_from_all_matches.json", kill_events_threshold=4, max_workers=4) #max_workers는 병렬 처리할 프로세스 수


### [4] ── 분(minute) 10 이상만 남기기 ──────────────────────
# ── 경로 설정 ────────────────────────────────────────────────
INPUT_JSON  = "data/kill_events_timeline_from_all_matches.json"
OUTPUT_JSON = "data/kill_events_timeline_filtered_10up.json"

# ── JSON 로드 ───────────────────────────────────────────────
with open(INPUT_JSON, "r", encoding="utf-8") as f:
    data = json.load(f)          # {"data": {match_id: [minutes...] }}

# ── 분(minute) 10 이상만 남기기 ────────────────────────────────
filtered = {
    mid: [m for m in minutes if m >= 10]
    for mid, minutes in data.get("data", {}).items()
    if any(m >= 10 for m in minutes)       # 10 이상이 하나도 없으면 통째로 제거
}

# ── 결과 저장 ───────────────────────────────────────────
os.makedirs(os.path.dirname(OUTPUT_JSON), exist_ok=True)
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump({"data": filtered}, f, ensure_ascii=False, indent=2)

print(f"저장 완료 → {OUTPUT_JSON}  (경기 수: {len(filtered)})")

### [5] 상위 N개 경기만 남도록 필터링
TOP_KILL_EVENTS_JSON = "data/top_kill_events_timeline_filtered_10up.json"
ra.save_top_matches_by_score(
    avg_score_csv_path=MATCH_AVGSCR_CSV_PATH,
    data_json_path=OUTPUT_JSON,
    output_path=TOP_KILL_EVENTS_JSON,
    top_n=1000
)


In [ ]:
### OPTION : [6] 여러 데스크탑에서 replay_scraper 돌리기 위한 json 파일 쪼개기
#  분할하여 저장할 경로
filtered_data_1_path = r"data/filltered_kill_events_timeline_filtered_10up_1.json"
filtered_data_2_path = r"data/filltered_kill_events_timeline_filtered_10up_2.json"

# 1) filtered_data.json 불러오기
with open(TOP_KILL_EVENTS_JSON, "r", encoding="utf-8") as f:
    original = json.load(f)
    
# 예: { "data": { "7641937851": [3,5], "7641940891": [3,4,6,17], ... } }
orig_data = original.get("data", {})

# 2) 두 개의 딕셔너리를 준비 (key들은 동일, value는 절반씩)
data_1 = {}
data_2 = {}

for match_id, arr in orig_data.items():
    if not isinstance(arr, list):
        # 혹시 list가 아니면 그냥 두 쪽 다 동일하게(또는 스킵) 할 수도 있음
        data_1[match_id] = arr
        data_2[match_id] = arr
        continue
    
    mid = len(arr) // 2  # 중앙 인덱스
    first_part = arr[:mid]
    second_part = arr[mid:]
    
    data_1[match_id] = first_part
    data_2[match_id] = second_part

# 3) 최종 저장 구조: { "data": data_1 }, { "data": data_2 }
final_1 = {"data": data_1}
final_2 = {"data": data_2}

with open(filtered_data_1_path, "w", encoding="utf-8") as f1:
    json.dump(final_1, f1, ensure_ascii=False, indent=2)

with open(filtered_data_2_path, "w", encoding="utf-8") as f2:
    json.dump(final_2, f2, ensure_ascii=False, indent=2)

print(f"Done. 두 파일이 생성되었습니다:\n{filtered_data_1_path}\n{filtered_data_2_path}")


Done. 두 파일이 생성되었습니다:
data/filltered_kill_events_timeline_filtered_10up_1.json
data/filltered_kill_events_timeline_filtered_10up_2.json
